In [1]:
import cv2
import numpy as np
import os

# Define the input and output directories
input_dir = "good"
output_dir = "split_discs"
os.makedirs(output_dir, exist_ok=True)

# Loop through each image in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".png"):  # Adjust the extension as needed
        image_path = os.path.join(input_dir, filename)
        
        # Extract just the [patient#]_t1/t2 part from the filename
        base_name = filename.split("original_with_boxes_")[-1].replace(".png", "")
        
        # Load and process the image
        image = cv2.imread(image_path)
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        # Apply the red color mask
        lower_red1 = np.array([0, 120, 70])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([170, 120, 70])
        upper_red2 = np.array([180, 255, 255])
        mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        mask = mask1 + mask2
        
        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Sort contours from top to bottom
        contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[1])
        
        # Save each red box as a separate image
        for i, contour in enumerate(contours):
            x, y, w, h = cv2.boundingRect(contour)
            red_box = image[y:y+h, x:x+w]
            output_path = os.path.join(output_dir, f"{base_name}_disc_{i + 1}.png")
            cv2.imwrite(output_path, red_box)

print("All red boxes have been extracted and saved as separate images in the 'split_discs' directory.")

All red boxes have been extracted and saved as separate images in the 'split_discs' directory.


In [5]:
import pandas as pd

# Load the CSV file
file_path = 'radiological_gradings.csv'
gradings_df = pd.read_csv(file_path)

# Filter the dataframe to include only rows for Patient 203
patient_203_df = gradings_df[gradings_df['Patient'] == 203]

# Define columns to include in the array aside from Patient ID and IVD label
grading_columns = ['Modic', 'UP endplate', 'LOW endplate', 'Spondylolisthesis', 
                   'Disc herniation', 'Disc narrowing', 'Disc bulging', 'Pfirrman grade']

# Filter for only the necessary columns
patient_203_data = patient_203_df[grading_columns].to_dict(orient='records')

# Associate each disc image with the grading data
disc_image_data = {f"disc_{i+1}.png": patient_203_data[i] for i in range(len(patient_203_data))}

# Display the final array mapping disc images to their grading information
print(disc_image_data)

{'disc_1.png': {'Modic': 0, 'UP endplate': 0, 'LOW endplate': 0, 'Spondylolisthesis': 0, 'Disc herniation': 0, 'Disc narrowing': 0, 'Disc bulging': 1, 'Pfirrman grade': 4}, 'disc_2.png': {'Modic': 0, 'UP endplate': 0, 'LOW endplate': 0, 'Spondylolisthesis': 0, 'Disc herniation': 0, 'Disc narrowing': 0, 'Disc bulging': 1, 'Pfirrman grade': 3}, 'disc_3.png': {'Modic': 0, 'UP endplate': 0, 'LOW endplate': 0, 'Spondylolisthesis': 0, 'Disc herniation': 0, 'Disc narrowing': 0, 'Disc bulging': 1, 'Pfirrman grade': 4}, 'disc_4.png': {'Modic': 0, 'UP endplate': 0, 'LOW endplate': 0, 'Spondylolisthesis': 0, 'Disc herniation': 0, 'Disc narrowing': 0, 'Disc bulging': 0, 'Pfirrman grade': 2}, 'disc_5.png': {'Modic': 0, 'UP endplate': 1, 'LOW endplate': 0, 'Spondylolisthesis': 0, 'Disc herniation': 0, 'Disc narrowing': 0, 'Disc bulging': 0, 'Pfirrman grade': 2}, 'disc_6.png': {'Modic': 0, 'UP endplate': 1, 'LOW endplate': 1, 'Spondylolisthesis': 0, 'Disc herniation': 0, 'Disc narrowing': 0, 'Disc bu

In [6]:
import cv2
import os
import pandas as pd

# Load the CSV file
csv_path = 'radiological_gradings.csv'
radiological_data = pd.read_csv(csv_path)

# Initialize lists for each label
pfirrman_grades = []
modic_types = []
up_endplates = []
low_endplates = []
disc_herniation = []
disc_narrowing = []
disc_bulging = []
spondylolisthesis = []
image_list = []  # For storing image data

# Directory containing the disc images
image_dir = "split_discs"

# Loop through each image in the directory
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        # Parse patient number and IVD label from the filename
        base_name = filename.replace(".png", "")
        
        # Split the filename and handle cases with additional parts
        parts = base_name.split("_")
        patient_id = int(parts[0])           # First part is always the patient ID
        scan_type = parts[-2]                # Second last part is the scan type (t1/t2)
        ivd_label = int(parts[-1].replace("disc", ""))  # Last part is the disc label
        
        # Find the corresponding row in the CSV
        row = radiological_data[(radiological_data['Patient'] == patient_id) &
                                (radiological_data['IVD label'] == ivd_label)]
        
        # Ensure the row exists and extract label values
        if not row.empty:
            # Append each label to its respective list
            modic_types.append(row['Modic'].values[0])
            up_endplates.append(row['UP endplate'].values[0])
            low_endplates.append(row['LOW endplate'].values[0])
            spondylolisthesis.append(row['Spondylolisthesis'].values[0])
            disc_herniation.append(row['Disc herniation'].values[0])
            disc_narrowing.append(row['Disc narrowing'].values[0])
            disc_bulging.append(row['Disc bulging'].values[0])
            pfirrman_grades.append(row['Pfirrman grade'].values[0])
            
            # Optionally load the image data if needed
            image_path = os.path.join(image_dir, filename)
            image = cv2.imread(image_path)
            image_list.append(image)  # Or process as necessary

print("Labels have been mapped to each image and stored in the corresponding lists.")

Labels have been mapped to each image and stored in the corresponding lists.
